In [1]:
import pandas as pd
import json

In [2]:
movies = pd.read_csv('./movies_dataset.csv')

In [3]:
movies_credits = pd.read_csv('./credits.csv')

In [4]:
def cantidad_filmaciones_mes(mes):
    meses_ingles = {
        'enero': 'January',
        'febrero': 'February',
        'marzo': 'March',
        'abril': 'April',
        'mayo': 'May',
        'junio': 'June',
        'julio': 'July',
        'agosto': 'August',
        'septiembre': 'September',
        'octubre': 'October',
        'noviembre': 'November',
        'diciembre': 'December'
    } #Mapeo de nombres de meses en español a nombres en inglés
    mes_en_ingles = meses_ingles.get(mes.lower()) #Convertimos el nombre del mes a minúsculas y buscar el nombre en inglés
    if mes_en_ingles is None:
        return "Mes no válido. Por favor, ingrese un mes válido en español."
    movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce') #Convertimos el campo release_date a tipo datetime para extraer el mes en inglés
    movies_mes = movies[movies['release_date'].dt.strftime('%B').str.lower() == mes_en_ingles.lower()] # Filtramos las películas que tienen release_date válida y coinciden con el mes dado
    cantidad = len(movies_mes) #Contamos la cantidad de películas
    
    return f"{cantidad} películas fueron estrenadas en el mes de {mes.capitalize()}."

In [5]:
cantidad_filmaciones_mes('Enero')

'5912 películas fueron estrenadas en el mes de Enero.'

In [6]:
def cantidad_filmaciones_dia(Dia):
    dias_ingles = {
        'lunes': 'Monday',
        'martes': 'Tuesday',
        'miércoles': 'Wednesday',
        'jueves': 'Thursday',
        'viernes': 'Friday',
        'sábado': 'Saturday',
        'domingo': 'Sunday',
    } # Mapeo de nombres de días en español a nombres en inglés
    
    dias_en_ingles = dias_ingles.get(Dia.lower()) # Convertimos el nombre del día a minúsculas y buscamos el nombre en inglés
    if dias_en_ingles is None:
        return "Día no válido. Por favor, ingrese un día válido en español."
    
    movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce') # Convertimos el campo release_date a tipo datetime para extraer el día de la semana en inglés
    movies_dia = movies[movies['release_date'].dt.strftime('%A').str.lower() == dias_en_ingles.lower()] # Filtramos las películas que tienen release_date válida y coinciden con el día dado
    cantidad = len(movies_dia) # Contamos la cantidad de películas
    
    return f"{cantidad} películas fueron estrenadas en el día {Dia.capitalize()}."

In [7]:
cantidad_filmaciones_dia('miércoles')

'7035 películas fueron estrenadas en el día Miércoles.'

In [8]:
def score_titulo(titulo_de_la_filmacion):
    movie = movies[movies['title'].str.lower() == titulo_de_la_filmacion.lower()] 
    
    if movie.empty:
        return f"No se encontró la película con el título '{titulo_de_la_filmacion}' en la base de datos."
    
    title = movie['title'].values[0]
    release_year = movie['release_year'].values[0]
    score = movie['popularity'].values[0]
    
    return f"La película '{title}' fue estrenada en el año {release_year} con un score/popularidad de {score}."

In [9]:
score_titulo('Toy Story')

"La película 'Toy Story' fue estrenada en el año 1995 con un score/popularidad de 21.946943."

In [10]:
def votos_titulo(titulo_de_la_filmacion):
    pelicula = movies[movies['title'].str.lower() == titulo_de_la_filmacion.lower()] #Buscamos la película por título
    
    if pelicula.empty:
        return f"No se encontró ninguna película con el título '{titulo_de_la_filmacion}'."
    
    votos_totales = pelicula['vote_count'].iloc[0]
    if votos_totales < 2000: #Verificamos si tiene al menos 2000 valoraciones
        return f"La película '{titulo_de_la_filmacion}' tiene menos de 2000 valoraciones ({votos_totales} valoraciones en total). No cumple con la condición requerida."
    
    promedio_votos = pelicula['vote_average'].iloc[0] #Calculamos el promedio de votos
    
    mensaje = f"La película '{titulo_de_la_filmacion}' fue estrenada en el año {pelicula['release_year'].iloc[0]}. "
    mensaje += f"La misma cuenta con un total de {votos_totales} valoraciones, con un promedio de {promedio_votos:.1f}."
    
    return mensaje


In [11]:
votos_titulo('Jumanji')

"La película 'Jumanji' fue estrenada en el año 1995. La misma cuenta con un total de 2413.0 valoraciones, con un promedio de 6.9."

In [12]:
def load_json(x):
    if isinstance(x, str):  
        try:
            return json.loads(x.replace("'", '"'))
        except json.JSONDecodeError:
            return []
    return []

In [13]:
movies_credits['cast'] = movies_credits['cast'].apply(load_json)

In [14]:
def get_actor(nombre_actor):
    actor_found = False
    total_films = 0
    total_return = 0.0

    for index, row in movies_credits.iterrows():
            for actor in row['cast']:
                if actor['name'] == nombre_actor:
                    actor_found = True
                    total_films += 1
                    total_return += row.get('return', 0.0)

    if not actor_found:
            return f"El actor {nombre_actor} no ha sido encontrado en el dataset."

    if total_films == 0:
            return f"El actor {nombre_actor} no ha participado en ninguna filmación."

    average_return = total_return / total_films
    return f"El actor {nombre_actor} ha participado en {total_films} filmaciones. Ha conseguido un retorno total de {total_return} con un promedio de {average_return} por filmación."

In [15]:
get_actor('Tom Hanks')

'El actor Tom Hanks ha participado en 15 filmaciones. Ha conseguido un retorno total de 0.0 con un promedio de 0.0 por filmación.'

In [16]:
def get_director(nombre_director):
    nombre_director = nombre_director.lower() #Convertimos el nombre del director a minúsculas para una comparación sin distinción entre mayúsculas y minúsculas
    peliculas_director = [] # Creamos una lista vacía para almacenar las películas del director

    for index, row in movies_credits.iterrows(): #Recorremos cada fila del dataframe movies_credits
        crew = load_json(row['crew'])  #Convertimos la cadena JSON de 'crew' a un diccionario usando load_json
        
        if isinstance(crew, list): #Verificamos si el director está en el crew y si es un director
            for member in crew:
                if member['job'].lower() == 'director' and nombre_director in member['name'].lower():
                    #Encontramos al director, obtener información de la película
                    movie_info = {
                        'titulo': row['title'],
                        'fecha_lanzamiento': row['release_date'],
                        'retorno': row['return'],
                        'costo': row['budget'],
                        'ganancia': row['revenue']
                    }
                    peliculas_director.append(movie_info)
                    break  #Salimos del bucle interno si encontramos al director
    
    if len(peliculas_director) > 0: #Verificamos si encontramos alguna película del director
        mensaje_retorno = f"El director {nombre_director.capitalize()} ha dirigido las siguientes películas:\n"
        for pelicula in peliculas_director:
            mensaje_retorno += f"Título: {pelicula['titulo']}, Fecha de Lanzamiento: {pelicula['fecha_lanzamiento']}, " \
                               f"Retorno: {pelicula['retorno']}, Costo: {pelicula['costo']}, Ganancia: {pelicula['ganancia']}\n"
    else:
        mensaje_retorno = f"El director {nombre_director.capitalize()} no ha dirigido ninguna película."
    
    return mensaje_retorno

In [18]:
get_director('Al Pacino')

'El director Al pacino no ha dirigido ninguna película.'